# XRT Blop Demo

For ophyd beamline setup see: 
- https://github.com/NSLS-II/blop/blob/main/src/blop/sim/xrt_beamline.py
- https://github.com/NSLS-II/blop/blob/main/src/blop/sim/xrt_kb_model.py

The picture below displays beam from geometric source propagating through a pair of toroidal mirrors focusing the beam on screen. Simulation of a KB setup.

![xrt_blop_layout_w.jpg](../_static/xrt_blop_layout_w.jpg)

In [ ]:
import datetime

import matplotlib.pyplot as plt
from bluesky.callbacks import best_effort
from bluesky.callbacks.tiled_writer import TiledWriter
from bluesky.run_engine import RunEngine
from ophyd.utils import make_dir_tree
from tiled.client import from_uri
from tiled.server import SimpleTiledServer
from blop.sim import beamline

tiled_server = SimpleTiledServer(readable_storage=[beamline.DECTECTOR_STORAGE])
tiled_client = from_uri(tiled_server.uri)
tiled_writer = TiledWriter(tiled_client)

# Set up RE enviornment
def setup_re_env(db_type="default", root_dir="/default/path"):
    RE = RunEngine({})
    bec = best_effort.BestEffortCallback()
    RE.subscribe(bec)
    RE.subscribe(tiled_writer)
    _ = make_dir_tree(datetime.datetime.now().year, base_path=root_dir)
    return {"RE": RE, "tiled": tiled_client, "bec": bec}


env = setup_re_env(db_type="temp", root_dir="/tmp/blop/sim/data")
globals().update(env)

bec.disable_plots()

Tiled version 0.1.dev2763+ge244fae


2025-06-04 11:33:18.840 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/table/partition/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/internal/events?partition=0 "HTTP/1.1 200 OK"
2025-06-04 11:33:19.198 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/array/block/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/external/bl_det_image?block=9%2C0%2C0&expected_shape=1%2C300%2C400 "HTTP/1.1 500 Internal Server Error"
2025-06-04 11:33:19.200 WARNING: Scheduled retry in 0.63 seconds due to HTTPStatusError("Server error '500 Internal Server Error' for url 'http://127.0.0.1:34083/api/v1/array/block/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/external/bl_det_image?block=9%2C0%2C0&expected_shape=1%2C300%2C400'\nFor more information, server admin can search server logs for correlation ID 49149663e5051c6d.") (attempt 1)
2025-06-04 11:33:19.206 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/array/block/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/external/bl_det_image?block=6%2C0%2C0

In [2]:
import time

from blop import DOF, Agent, Objective
from blop.digestion import beam_stats_digestion
from blop.sim.xrt_beamline import Beamline

/home/jmoylan1/miniconda3/envs/test_tiled_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
plt.ion()

h_opt = 0
dh = 5

R1, dR1 = 40000, 10000
R2, dR2 = 20000, 10000

In [4]:
beamline = Beamline(name="bl")
time.sleep(1)
dofs = [
    DOF(description="KBV R",
        device=beamline.kbv_dsv,
        search_domain=(R1 - dR1, R1 + dR1)),
    DOF(description="KBH R",
        device=beamline.kbh_dsh,
        search_domain=(R2 - dR2, R2 + dR2)),

]

In [5]:
objectives = [
    Objective(name="bl_det_sum",
              target="max",
              transform="log",
              trust_domain=(20, 1e12)),

    Objective(name="bl_det_wid_x",
              target="min",
              transform="log",
              # trust_domain=(0, 1e12),
              latent_groups=[("bl_kbh_dsh", "bl_kbv_dsv")]),
    Objective(name="bl_det_wid_y",
              target="min",
              transform="log",
              # trust_domain=(0, 1e12),
              latent_groups=[("bl_kbh_dsh", "bl_kbv_dsv")]),
]

In [6]:
# import json
# class JSONWriter:
#     """Writer for a JSON array"""

#     def __init__(self, filepath):
#         self.file = open(filepath, "w")
#         self.file.write("[\n")

#     def __call__(self, name, doc):
#         json.dump({"name": name, "doc": doc}, self.file)
#         if name == "stop":
#             self.file.write("\n]")
#             self.file.close()
#         else:
#             self.file.write(",\n")

# wr = JSONWriter('/tmp/blop/sim/test_tiled_writer1.json')
# RE.subscribe(wr)

In [7]:
agent = Agent(
    dofs=dofs,
    objectives=objectives,
    detectors=[beamline.det],
    digestion=beam_stats_digestion,
    digestion_kwargs={"image_key": "bl_det_image"},
    verbose=True,
    tiled=tiled,
    tolerate_acquisition_errors=False,
    enforce_all_objectives_valid=True,
    train_every=3,
)

In [8]:
RE(agent.learn("qr", n=16))

2025-06-04 11:33:16.902 INFO: Executing plan <generator object Agent.learn at 0x74b80c2c6a70>
2025-06-04 11:33:16.903 INFO: Change state on <bluesky.run_engine.RunEngine object at 0x74b8c8fb19d0> from 'idle' -> 'running'
2025-06-04 11:33:16.904 INFO: running iteration 1 / 1


image_shape: (300, 400)
(1, 300, 400)


Transient Scan ID: 1     Time: 2025-06-04 11:33:16
Persistent Unique Scan ID: '22091bcc-c010-45a7-b8b2-76e0b49eb8de'


2025-06-04 11:33:16.940 INFO: HTTP Request: POST http://127.0.0.1:34083/api/v1/metadata/ "HTTP/1.1 200 OK"


New stream: 'primary'
+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
|   seq_num |       time | bl_kbv_dsv | bl_kbh_dsh | bl_det_sum | bl_det_cen_x | bl_det_cen_y | bl_det_wid_x | bl_det_wid_y |
+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+


2025-06-04 11:33:16.982 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/search//22091bcc-c010-45a7-b8b2-76e0b49eb8de?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-04 11:33:16.994 INFO: HTTP Request: POST http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de "HTTP/1.1 200 OK"
2025-06-04 11:33:17.006 INFO: HTTP Request: POST http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary "HTTP/1.1 200 OK"
2025-06-04 11:33:17.017 INFO: HTTP Request: POST http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary "HTTP/1.1 200 OK"
2025-06-04 11:33:17.029 INFO: HTTP Request: POST http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary "HTTP/1.1 200 OK"
2025-06-04 11:33:17.044 INFO: HTTP Request: PATCH http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary "HTTP/1.1 200 OK"
2025-06-04 11:33:17.052 INFO: HTTP Request: GET http://127.0.0.1:34083/ap

...

|        16 | 11:33:18.6 |  48383.842 |  10360.336 |      2.117 |      187.032 |      140.754 |      338.747 |      268.054 |


2025-06-04 11:33:18.662 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/internal "HTTP/1.1 200 OK"
2025-06-04 11:33:18.670 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/search/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/internal?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-04 11:33:18.679 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/metadata/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/internal/events "HTTP/1.1 200 OK"
2025-06-04 11:33:18.690 INFO: HTTP Request: PATCH http://127.0.0.1:34083/api/v1/table/partition/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/internal/events?partition=0 "HTTP/1.1 200 OK"


+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
generator list_scan ['22091bcc'] (scan num: 1)





2025-06-04 11:33:18.803 INFO: HTTP Request: PATCH http://127.0.0.1:34083/api/v1/metadata//22091bcc-c010-45a7-b8b2-76e0b49eb8de "HTTP/1.1 200 OK"
2025-06-04 11:33:18.813 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/metadata/22091bcc-c010-45a7-b8b2-76e0b49eb8de "HTTP/1.1 200 OK"
2025-06-04 11:33:18.822 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/metadata/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/internal/events "HTTP/1.1 200 OK"
2025-06-04 11:33:18.850 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/metadata/22091bcc-c010-45a7-b8b2-76e0b49eb8de "HTTP/1.1 200 OK"
2025-06-04 11:33:18.861 INFO: HTTP Request: GET http://127.0.0.1:34083/api/v1/metadata/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/external/bl_det_image "HTTP/1.1 200 OK"
2025-06-04 11:33:42.133 ERROR: Run aborted
Traceback (most recent call last):
  File "/home/jmoylan1/miniconda3/envs/test_tiled_new/lib/python3.11/site-packages/bluesky/run_engine.py", line 1609, in _run
    msg = self._plan_stack[-1

HTTPStatusError: Server error '500 Internal Server Error' for url 'http://127.0.0.1:34083/api/v1/array/block/22091bcc-c010-45a7-b8b2-76e0b49eb8de/primary/external/bl_det_image?block=3%2C0%2C0&expected_shape=1%2C300%2C400'
For more information, server admin can search server logs for correlation ID 6042179af80f5e80.

In [ ]:
# RE(agent.learn("qr", n=4, iterations=4))


In [ ]:
agent.plot_objectives()